# Demo Deep Belief Network para identificar TIPOS de ANIMALES


0) Instalar paquete de DBN: 
( para más info ver https://github.com/albertbup/deep-belief-network/blob/master/README.md )

In [ ]:
pip install git+git://github.com/albertbup/deep-belief-network.git

1) Cargar librerías:

In [ ]:
#@title Librerías a usar

# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

from dbn.tensorflow import SupervisedDBNClassification
import numpy as np
import pandas as pd
from numpy.random import RandomState

from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt

print ("Librerías cargadas.")

2) Configurar los datos cargados (se debe definir de acuerdo a los datos):

In [ ]:
## selección de los parámetros 

#@markdown ### Parámetros de archivo de datos:
archivo_datos = '/datos/animales.csv'  #@param {type:"string"}
atributo_clase = 'TIPO' #@param {type:"string"}
nombre_clases = 'MAMIFERO, AVE, REPTIL, PEZ, ANFIBIO, INSECTO, INVERTEBRADO' #@param {type:"string"}

proporcion_porcentaje_datos_entrenamiento = 69  #@param {type:"integer"}

## aplicación de los parámetros elegidos

# define nombre atributo de CLASE para ejemplo ANIMALES
ClassAttributeName = atributo_clase

# define valores de clases para ejemplo ANIMALES
CLASES = [ 'na' ]
for val in nombre_clases.split(','):
  CLASES.append( val )

print("Configuración definida de ", archivo_datos)
print("Atributo clase: ", ClassAttributeName, ": ", CLASES)


3) Cargar CSV con datos a procesar y preparar datos para entrenar y probar (ya separados):

In [ ]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = 'gdrive/My Drive/Colab Notebooks/IA-RNA-master'  #@param {type:"string"}

In [ ]:
#@title Preparar datos de entrenamiento

# Carga los datos del CSV y muestra los primeros
df = pd.read_csv(path + archivo_datos)
df.head()

# separa al azar con proporción indicada
rng = RandomState()

if proporcion_porcentaje_datos_entrenamiento>100:
  propTrain = 1
elif proporcion_porcentaje_datos_entrenamiento<1:
  propTrain = 0.1
else:
  propTrain = proporcion_porcentaje_datos_entrenamiento/100
train = df.sample(frac=propTrain, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

print("Datos Originales ", df.shape)
print("- Datos para Entrenar ", train.shape)
print("- Datos para Probar ", test.shape)

# define datos de entrada y salida para entrenamiento
X_train = np.array(train.drop([ClassAttributeName], axis=1))
Y_train = np.array(train[ClassAttributeName])

# define datos de entrada y salida para testing
X_test = np.array(test.drop([ClassAttributeName], axis=1))
Y_test = np.array(test[ClassAttributeName])

print("Datos Originales ", len(X_train)+len(X_test))
print("- Datos para Entrenar ", len(X_train))
print("- Datos para Probar ", len(X_test))

4) Especificar el modelo para la DBN:

In [ ]:
#@title Establecer el modelo

classifier = SupervisedDBNClassification(hidden_layers_structure = [256, 128],
learning_rate_rbm=0.05,
learning_rate=0.1,
n_epochs_rbm=20,
n_iter_backprop=200,
batch_size=32,
activation_function='relu',
dropout_p=0.2)

print(classifier)

5) Entrenar el modelo de la DBN:

In [ ]:
#@title Entrenar
classifier.fit(X_train, Y_train)

In [ ]:
#@title Evaluar el modelo con datos de entrenamiento

# ejecuta el  modelo
Y_pred = classifier.predict(X_train)

# calcula la exactitud
print('\nExactitud: %f' % accuracy_score(Y_train, Y_pred))

# muestra resultados de la predicción
classPreds = []
classReal = []
cantOK = 0
cantError = 0
print("\n Resultados: ")
for pred_class_id, expec in zip(Y_pred, Y_train):
    
    classPreds.append(CLASES[pred_class_id])
    classReal.append(CLASES[expec])

    if (pred_class_id == expec): 
      res = ""
      cantOK += 1
    else: 
      res = "!"
      cantError += 1

    print('Clase predecida es "{}"[{}], la correcta es "{}"[{}]{} '.format(
        CLASES[pred_class_id], pred_class_id, CLASES[expec], expec, res))

print('=== Total Ejemplos: {}, con {} predicciones ok y {} errores.'.format(cantOK+cantError, cantOK, cantError))

# gráfico de comparación
plt.title('Gráfico de Confusión')
plt.xlabel('Real')
plt.ylabel('DBN')
plt.scatter(classReal, classPreds)

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión: ')
cm = confusion_matrix(classReal, classPreds, labels=CLASES)
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in CLASES], 
    columns=['p:{:}'.format(x) for x in CLASES]
  )
print(cmtx)
print("\n")


6) Evaluar el modelo de la RNA entrenado:

In [ ]:
#@title Evaluar el modelo con datos de prueba

# ejecuta el  modelo
Y_pred = classifier.predict(X_test)

# calcula la exactitud
print('\nExactitud: %f' % accuracy_score(Y_test, Y_pred))

# muestra resultados de la predicción
classPreds = []
classReal = []
cantOK = 0
cantError = 0
print("\n Resultados: ")
for pred_class_id, expec in zip(Y_pred, Y_test):
    
    classPreds.append(CLASES[pred_class_id])
    classReal.append(CLASES[expec])

    if (pred_class_id == expec): 
      res = ""
      cantOK += 1
    else: 
      res = "!"
      cantError += 1

    print('Clase predecida es "{}"[{}], la correcta es "{}"[{}]{} '.format(
        CLASES[pred_class_id], pred_class_id, CLASES[expec], expec, res))

print('=== Total Ejemplos: {}, con {} predicciones ok y {} errores.'.format(cantOK+cantError, cantOK, cantError))

# gráfico de comparación
plt.title('Gráfico de Confusión')
plt.xlabel('Real')
plt.ylabel('DBN')
plt.scatter(classReal, classPreds)

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión: ')
cm = confusion_matrix(classReal, classPreds, labels=CLASES)
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in CLASES], 
    columns=['p:{:}'.format(x) for x in CLASES]
  )
print(cmtx)
print("\n")
